In [2]:
import numpy as np
import pandas as pd

np.random.seed(12)

In [22]:
np.random.randint(0, 5)

1

In [12]:
# Let's say we have 5 stations and each station has 2 DSPs
stations = list(range(1, 6))
print(stations)
dsp = list(range(11, 21))
print(dsp)

[1, 2, 3, 4, 5]
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [3]:
volume_forecast = dict(
    zip(stations, np.random.normal(loc=1000, scale=50, size=5).clip(800.0, 1200.0))
)

spr_forecast = dict(
    zip(stations, np.random.normal(loc=25.0, scale=10.0, size=5).clip(15.0, 50.0))
)
routes_forecast = dict(
    zip(dsp, np.random.normal(loc=30.0, scale=20.0, size=10).clip(10.0, 70.0))
)

data = {"week": np.repeat(50, 10), "stations": stations + stations, "dsp": dsp}
station_dsp = pd.DataFrame(data=data)
station_dsp = station_dsp.sort_values(by=["stations", "dsp"]).reset_index(drop=True)
station_dsp["volume_forecast"] = (
    station_dsp["stations"].map(volume_forecast).astype("int32")
)
station_dsp["spr_forecast"] = station_dsp["stations"].map(spr_forecast).astype("int32")
station_dsp["routes_demand"] = (
    station_dsp.volume_forecast / station_dsp.spr_forecast
).astype("int32")
station_dsp["dsp_capacity"] = station_dsp["dsp"].map(routes_forecast).astype("int32")
station_dsp["volume_share"] = [0.3, 0.7, 0.6, 0.4, 0.5, 0.5, 0.6, 0.4, 0.8, 0.2]
station_dsp["dsp_routes_demand"] = station_dsp.routes_demand * station_dsp.volume_share
station_dsp["is_elligible_target"] = np.where(
    station_dsp.dsp_routes_demand < station_dsp.dsp_capacity, 1, 0
)
station_cap = (
    station_dsp.groupby(["stations"])["dsp_capacity"]
    .sum()
    .astype("int32")
    .reset_index()
)
station_cap.rename(columns={"dsp_capacity": "station_capacity"}, inplace=True)
station_dsp = station_dsp.merge(station_cap, on="stations", how="left")
station_dsp["cap_gap_station"] = (
    station_dsp["station_capacity"] - station_dsp["routes_demand"]
)
station_dsp["is_under_solved"] = np.where(station_dsp.cap_gap_station < 0, 1, 0)
station_dsp["is_elligible_target"] = (
    station_dsp.is_elligible_target - station_dsp.is_under_solved
) * station_dsp.is_elligible_target
station_dsp

,week,stations,dsp,volume_forecast,spr_forecast,routes_demand,dsp_capacity,volume_share,dsp_routes_demand,is_elligible_target,station_capacity,cap_gap_station,is_under_solved
0,50,1,11,1023,15,68,18,0.3,20.4,0,45,-23,1
1,50,1,16,1023,15,68,27,0.7,47.6,0,45,-23,1
2,50,2,12,965,25,38,39,0.6,22.8,1,89,51,0
3,50,2,17,965,25,38,50,0.4,15.2,1,89,51,0
4,50,3,13,1012,23,44,51,0.5,22.0,1,62,18,0
5,50,3,18,1012,23,44,11,0.5,22.0,0,62,18,0
6,50,4,14,914,16,57,10,0.6,34.2,0,20,-37,1
7,50,4,19,914,16,57,10,0.4,22.8,0,20,-37,1
8,50,5,15,1037,50,20,56,0.8,16.0,1,110,90,0
9,50,5,20,1037,50,20,54,0.2,4.0,1,110,90,0


In [4]:
package_cost = np.sum(
    np.abs(
        station_dsp.cap_gap_station
        * station_dsp.spr_forecast
        * station_dsp.is_under_solved
    )
    / 2
)
excess_capacity_cost = 0.1 * np.sum(
    (
        station_dsp[station_dsp.is_under_solved == 0]["cap_gap_station"]
        * station_dsp.spr_forecast
    )
    / 2
)
print(f"current cost of system = {package_cost} + {excess_capacity_cost}")

current cost of system = 937.0 + 618.9000000000001


In [6]:
oversolved_stations = station_dsp[station_dsp.is_under_solved == 0].stations.unique()
undersolved_stations = station_dsp[station_dsp.is_under_solved == 1].stations.unique()
print(oversolved_stations, undersolved_stations)

[2 3 5] [1 4]


In [11]:
elligible_dsps = station_dsp[station_dsp.is_elligible_target == 1].dsp.unique()
print(elligible_dsps)

popups_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=15, scale=10.0, size=len(elligible_dsps)).clip(5.0, 30.0),
    )
)

pinnacle_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=20, scale=10.0, size=len(elligible_dsps)).clip(5.0, 30.0),
    )
)

transfer_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=30, scale=10.0, size=len(elligible_dsps)).clip(15.0, 50.0),
    )
)

new_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=10, scale=10.0, size=len(elligible_dsps)).clip(0.0, 20.0),
    )
)

data = {"dsp": elligible_dsps}
capacity_add = pd.DataFrame(data)

capacity_add["popup"] = capacity_add["dsp"].map(popups_forecast)
capacity_add["pinnacle"] = capacity_add["dsp"].map(pinnacle_forecast)
capacity_add["transfer"] = capacity_add["dsp"].map(transfer_forecast)
capacity_add["new"] = capacity_add["dsp"].map(new_forecast)
capacity_add

[12 17 13 15 20]


,dsp,popup,pinnacle,transfer,new
0,12,14.485850,29.435754,32.227195,8.947320
1,17,16.597877,23.576442,15.000000,15.120222
2,13,7.837364,19.165508,40.292112,20.000000
3,15,15.505228,26.778061,18.337412,0.000000
4,20,13.566626,25.560604,19.904383,20.000000


In [16]:
n_elligible_dsps = station_dsp[station_dsp.is_elligible_target == 0].dsp.unique()
print(n_elligible_dsps)

popups_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=3.0, scale=1.0, size=len(n_elligible_dsps)).clip(0.0, 5.0),
    )
)

pinnacle_forecast = dict(
    zip(
        elligible_dsps,
        np.random.normal(loc=3.0, scale=1.0, size=len(elligible_dsps)).clip(0.0, 5.0),
    )
)

data = {"dsp": elligible_dsps}
capacity_sub = pd.DataFrame(data)

capacity_sub["popup"] = capacity_sub["dsp"].map(popups_forecast)
capacity_sub["pinnacle"] = capacity_sub["dsp"].map(pinnacle_forecast)
capacity_sub["transfer"] = capacity_sub["dsp"].map(transfer_forecast)
capacity_sub["new"] = 0
capacity_sub

[11 16 18 14 19]


,dsp,popup,pinnacle,transfer,new
0,12,4.636463,1.660611,32.227195,0
1,17,2.538605,1.310347,15.000000,0
2,13,2.798638,2.800673,40.292112,0
3,15,2.428183,3.257773,18.337412,0
4,20,2.396701,4.828821,19.904383,0


In [15]:
scenario = np.array(capacity_add[["popup", "pinnacle", "transfer", "new"]]).flatten()
oversolved = pd.DataFrame({"station": oversolved_stations})

(20,)

In [50]:
x = np.array(np.repeat([1, 0, 0, 0], 5)).reshape(1, 20)
x

array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [28]:
A_ub = np.sum(x, axis=1).reshape(5, 1)
A_ub

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [25]:
A = np.repeat(1, 5).reshape(5, 1)
A

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [34]:
map_x = np.repeat(x.flatten().reshape(1, 20), 2).reshape(len(undersolved_stations), 20)
map_x

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [38]:
b_ub = np.sum(map_x, axis=0)
b_ub

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [40]:
B = np.repeat(1, 20).reshape(1, 20)
B

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [47]:
bounds = list(zip(np.repeat(0, 20), np.repeat(1, 20)))
bounds

[(0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1)]

In [19]:
# c = -np.array(list(price.values()))
# c

In [20]:
# A_ub = np.array([np.array(list(w.values())), np.array(list(v.values()))])

# A_ub

In [ ]:
b_ub = np.array([kw, kv])
b_ub

In [ ]:
bounds = [
    (0, 1),
] * 10
bounds

In [ ]:
from scipy.optimize import linprog

# Obtain solution
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

print(sol)

In [ ]:
integrality_vector = np.full(c.shape[0], 1)
print(integrality_vector)
sol_int = linprog(
    c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector
)
print(sol_int)